In [ ]:
import gc
import os
import json
import time
from PIL import Image
from tqdm import tqdm

import torch
from google.colab import drive
from transformers import TrOCRProcessor, VisionEncoderDecoderModel

# Memory management optimization
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

# Google Drive connection
drive.mount('/content/drive')


def is_image_file(filename):
    return filename.lower().endswith(('.png', '.jpg', '.jpeg'))


def run_trocr_on_existing_json(image_root, input_json_path, model_path = "microsoft/trocr-large-printed"):
    """Appends TrOCR result to existing JSON file."""

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"Using device: {device}")

    print("Initializing TrOCR model...")

    model_name = model_path.split("/")[-1].lower()

    # Initialize processor and model
    processor = TrOCRProcessor.from_pretrained(model_path)
    model = VisionEncoderDecoderModel.from_pretrained(model_path).cuda()

    # Load current JSON
    with open(input_json_path, 'r', encoding='utf-8') as f:
        data = json.load(f)

    start_time = time.time()

    for filename in tqdm([k for k in data if not k.startswith("_")], desc="OCR process"):
        for root, _, files in os.walk(image_root):
            if 'text_category' in root:
                continue
            if filename in files:
                img_path = os.path.join(root, filename)
                try:
                    image = Image.open(img_path).convert('RGB')

                    # Process image
                    pixel_values = processor(images=image, return_tensors="pt").pixel_values.cuda()

                    # Generate text
                    with torch.inference_mode():
                        generated_ids = model.generate(pixel_values)
                        extracted_text = processor.batch_decode(generated_ids, skip_special_tokens=True)[0]

                    # Clean up
                    del pixel_values, generated_ids
                    torch.cuda.empty_cache()
                    torch.cuda.synchronize()

                    # Write OCR results
                    data[filename]["models"][model_name] = {
                        "prediction": extracted_text,
                        "cer": None,
                        "wer": None
                    }

                except Exception as e:
                    print(f"{filename} error: {str(e)}")
                break  



    elapsed = round(time.time() - start_time,2)
    print(f"\n🕒 OCR tamamlandı: {elapsed:.2f} saniye")

    meta = data.get("_meta", {})
    processing_times = meta.get("processing_times", {})
    processing_times[model_name] = elapsed
    meta["processing_times"] = processing_times
    data["_meta"] = meta

    # Güncellenmiş JSON'u kaydet
    with open(input_json_path, 'w', encoding='utf-8') as f:
        json.dump(data, f, ensure_ascii=False, indent=2)

    print(f" Updated JSON saved: {input_json_path}")
    return data




input_json_path = '/content/drive/MyDrive/nutuk/benchmark/converted_data.json'
image_root = '/content/drive/MyDrive/nutuk/benchmark/'



# start OCR process
updated = run_trocr_on_existing_json(image_root, input_json_path)